Unzip the Data

In [ ]:
import zipfile
path_to_zip_file = 'data.zip'
directory_to_extract_to = './data/'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

Process raw ecg data, clean, R peak detection, outlier r peak removal, feature computation and standardization

In [30]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from tomkin import detect_rpeak
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from outlier_calculation import Quality,compute_outlier_ecg
# from hrvanalysis import remove_ectopic_beats
from joblib import Parallel,delayed
from data_quality import ECGQualityCalculation
from joblib import delayed,Parallel
from copy import deepcopy
from ecg import ecg_feature_computation
from scipy import interpolate
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler,RobustScaler,QuantileTransformer
import gzip
from scipy import stats

def get_interpolated(aclx,acly,aclz):
    time_array = aclx[:,1].reshape(-1,1)
    aclxyz = np.concatenate([time_array,time_array,time_array,time_array],axis=1)
    f = interpolate.interp1d(aclx[:,1],aclx[:,0],fill_value='extrapolate')
    aclxyz[:,1] = f(aclxyz[:,0])
    f = interpolate.interp1d(acly[:,1],acly[:,0],fill_value='extrapolate')
    aclxyz[:,2] = f(aclxyz[:,0])
    f = interpolate.interp1d(aclz[:,1],aclz[:,0],fill_value='extrapolate')
    aclxyz[:,3] = f(aclxyz[:,0])
    return aclxyz


def get_clean_ecg(ecg_data):
    final_data = np.zeros((0,3))
    if len(ecg_data)==0:
        return final_data
    test_object = ECGQualityCalculation()
    start_ts = ecg_data[0,0]
    final_data = np.zeros((0,3))
    while start_ts<ecg_data[-1,0]:
        index = np.where((ecg_data[:,0]>=start_ts)&(ecg_data[:,0]<start_ts+3000))[0]
        temp_data = ecg_data[index,2]
        if test_object.current_quality(temp_data)==1:
            final_data = np.concatenate((final_data,ecg_data[index,:]))
        start_ts = start_ts + 3000
    return final_data


def get_hr(ecg_data):
#     try:
    rpeaks = detect_rpeak(ecg_data[:,2],64)
    rpeak_ts = ecg_data[rpeaks,0]
    ecg_rr = np.zeros((len(rpeaks)-1,2))
    ecg_rr_ts = np.array(rpeak_ts)[1:]
    ecg_rr_sample = np.array(np.diff(rpeak_ts))
    index = np.where((ecg_rr_sample>=300)&(ecg_rr_sample<=2000))[0]
    ecg_rr_ts = ecg_rr_ts[index]
    ecg_rr_sam = ecg_rr_sample[index]
#     rr = remove_ectopic_beats(ecg_rr_sam)
    rr = ecg_rr_sam
    ecg_rr_sam = ecg_rr_sam[~np.isnan(rr)]
    ecg_rr_ts = ecg_rr_ts[~np.isnan(rr)]
    outlier = compute_outlier_ecg(ecg_rr_ts/1000,ecg_rr_sam/1000)
    ind1 = []
    for ind,tup in enumerate(outlier):
        if tup[1]==Quality.ACCEPTABLE:
            ind1.append(ind)
    ind1 = np.array(ind1)
    if len(ind1)<100:
        return [],[]
    ecg_rr_ts = ecg_rr_ts[ind1]
    ecg_rr_sam = ecg_rr_sam[ind1]
    return ecg_rr_ts,ecg_rr_sam
#     except Exception as e:
#         print(e)



def get_2sec_rr(data):
    ts_array = np.arange(data[0,0],data[-1,0],2000)
    window_col = []
    for t in ts_array:
        index = np.where((data[:,0]>t-2500)&(data[:,0]<=t+2500))[0]
        if len(index)==0:
            continue
        window_col.append(np.array([t,np.mean(data[index,1])]))
    return np.array(window_col)

def get_windows(data,window_size=10,offset=10,fs=1):
    ts_array = np.arange(data[0,0],data[-1,0],offset*1000)
    window_col = []
    for t in ts_array:
        index = np.where((data[:,0]>t-window_size*1000/2)&(data[:,0]<=t+window_size*1000/2))[0]
        if len(index)<10:
            continue
        window = data[index,:]
        window[:,1] = stats.mstats.winsorize(window[:,1],limits=.02)
        window_col.append(data[index,:])
    return window_col




def get_std_chest(window,start=1,end=4):
    return np.array([np.mean(window[:,0]),np.sqrt(np.sum(np.power(np.std(window[:,start:end],axis=0),2)))])


def filter_ecg_windows(ecg_windows,acl_std):
    final_ecg_windows = []
    for window in ecg_windows:
        index = np.where((acl_std[:,0]>window[0,0])&(acl_std[:,0]<window[-1,0]))[0]
        if len(index)==0:
            continue
        window_temp = acl_std[index,1].reshape(-1)
        if len(window_temp[window_temp>.21])/len(window_temp) > .5:
            continue
        final_ecg_windows.append(window)
    return final_ecg_windows


import numpy as np

class OnlineVariance(object):
    """
    Welford's algorithm computes the sample variance incrementally.
    """

    def __init__(self, iterable=None, ddof=1):
        self.ddof, self.n, self.mean, self.M2 = ddof, 0, 0.0, 0.0
        if iterable is not None:
            for datum in iterable:
                self.include(datum)

    def include(self, datum):
        self.n += 1
        self.delta = datum - self.mean
        self.mean += self.delta / self.n
        self.M2 += self.delta * (datum - self.mean)

    @property
    def variance(self):
        return self.M2 / (self.n - self.ddof)

    @property
    def std(self):
        return np.sqrt(self.variance)
    
    
    
# from __future__ import division
import numpy


class incre_std_avg:
    '''
    Incremental calculation of the average value, standard deviation, and variance of massive data
         1. Data
         obj.avg is the average
         obj.std is the standard deviation
         obj.n is the number of data
         When the object is initialized, you need to specify the historical average, historical standard deviation, and the number of historical data (the initial data set is empty if you don't need to fill in)
         2. Method
         The obj.incre_in_value() method passes in a new data to be calculated, performs incremental calculations, and obtains new avg, std and n (for massive data, please bring each new parameter loop into this method)
    '''

    def __init__(self, h_avg=0, h_std=0, n=0):
        self.avg = h_avg
        self.std = h_std
        self.n = n

    def incre_in_value(self, value):
        incre_avg = (self.n*self.avg+value)/(self.n+1)
        incre_std = numpy.sqrt((self.n*(self.std**2+(incre_avg-self.avg)
                                        ** 2)+(incre_avg-value)**2)/(self.n+1))
        self.avg = incre_avg
        self.std = incre_std
        self.n += 1


# if __name__ == "__main__":
#     c = incre_std_avg()
#     c.incre_in_value(0.05)
#     print c.avg
#     print c.std
#     print c.n
#     c.incre_in_value(0.02)
#     c.incre_in_list([0.5, 0.2, 0.3])
#     print c.avg
#     print c.std
#     print c.n





path = './data/'
participants = [path + f +'/' for f in os.listdir(path) if f[0]=='S']
for f in participants:
    if 'ecg.txt.gz' not in os.listdir(f):
        continue
    st = 0
    et = 0 
    with gzip.open(f+'stress_marks.txt.gz', 'r') as file:
        for line in file.readlines():
            line = line.decode('utf8').strip()
            parts = [x.strip() for x in line.split(',')]
            label = parts[0]
            if label[:2] in ['c1']:
                st = np.int64(parts[2])
                et = np.int64(parts[3])
#     aclx = pd.read_csv(f +'accelx.txt.gz', compression='gzip',
#                           sep=' ',header=None).values
#     acly = pd.read_csv(f +'accely.txt.gz', compression='gzip',
#                           sep=' ',header=None).values
#     aclz = pd.read_csv(f +'accelz.txt.gz', compression='gzip',
#                           sep=' ',header=None).values
#     acl_all = get_interpolated(aclx,acly,aclz)
#     aclx = ((3*acl_all[:,1].reshape(-1)/4095) - 1.5) / 0.3
#     acly = ((3*acl_all[:,2].reshape(-1)/4095) - 1.5) / 0.3
#     aclz = ((3*acl_all[:,3].reshape(-1)/4095) - 1.5) / 0.3
#     aclxyz = np.concatenate([acl_all[:,0].reshape(-1,1),aclx.reshape(-1,1),acly.reshape(-1,1),aclz.reshape(-1,1)],axis=1)
#     acl_windows = get_windows(aclxyz,window_size=10,offset=10,fs=15)
#     acl_std = np.array([get_std_chest(window) for window in acl_windows])
#     ecg_temp = pd.read_csv(f +'ecg.txt.gz', compression='gzip',
#                           sep=' ',header=None).values
#     print(ecg_temp.shape,f)
#     ecg = np.zeros((ecg_temp.shape[0],ecg_temp.shape[1]+1))
#     ecg[:,0],ecg[:,2] = ecg_temp[:,1],ecg_temp[:,0]
#     ecg_rr_ts,ecg_rr_sam = get_hr(ecg)
#     if len(ecg_rr_sam)<100:
#         continue
#     print(ecg_rr_ts.shape,f)
#     ecg_rr = np.zeros((len(ecg_rr_ts),2))
#     ecg_rr[:,0] = ecg_rr_ts
#     ecg_rr[:,1] = ecg_rr_sam
#     pickle.dump(ecg_rr,open(f+'ecg_rr.p','wb'))
    
    if os.path.isfile(f+'ecg_rr.p') and st>0:
        ecg_rr = pickle.load(open(f+'ecg_rr.p','rb'))
        ecg_rr = get_2sec_rr(ecg_rr)
        ecg_rr[:,1] = 60000/ecg_rr[:,1]
        ecg_windows = get_windows(ecg_rr,window_size=60,offset=30,fs=1)
        final_ecg_windows = ecg_windows
        ecg_features = np.array([np.array([window[0,0],window[-1,0]]+ecg_feature_computation(window[:,0],window[:,1])[:-4]) for window in final_ecg_windows])
        ecg_features = ecg_features[ecg_features[:,0].argsort(),:]
        n_features = 7
        mean_std_class_array = [incre_std_avg() for i in range(n_features)]
        N = 3
        final_features = []
        for i in range(1,ecg_features.shape[0]):
            feature_array = ecg_features[i]
            st,et,features = feature_array[0],feature_array[1],np.array(feature_array[2:])
            normalized_feature_array = [st,et]
            for k,value in enumerate(features):
                mean_std_class_array[k].incre_in_value(value)
                if i<N:
                    continue
                avg, std = mean_std_class_array[k].avg,mean_std_class_array[k].std
                value = (value-avg)/std
                normalized_feature_array.append(value)
            if len(normalized_feature_array)==n_features+2:
                final_features.append(np.array(normalized_feature_array))

        ecg_features_normalized = np.array(final_features)
        print(len(ecg_windows),len(final_ecg_windows),ecg_features.shape,path,f,ecg_features.shape,ecg_features_normalized.shape)
        pickle.dump(ecg_features_normalized,open(f+'features22norm.p','wb'))

240 240 (240, 9) ./data/ ./data/SI01/ (240, 9) (237, 9)
287 287 (287, 9) ./data/ ./data/SI02/ (287, 9) (284, 9)
256 256 (256, 9) ./data/ ./data/SI03/ (256, 9) (253, 9)
244 244 (244, 9) ./data/ ./data/SI04/ (244, 9) (241, 9)
263 263 (263, 9) ./data/ ./data/SI06/ (263, 9) (260, 9)
279 279 (279, 9) ./data/ ./data/SI07/ (279, 9) (276, 9)
127 127 (127, 9) ./data/ ./data/SI08/ (127, 9) (124, 9)
271 271 (271, 9) ./data/ ./data/SI10/ (271, 9) (268, 9)
261 261 (261, 9) ./data/ ./data/SI12/ (261, 9) (258, 9)
265 265 (265, 9) ./data/ ./data/SI13/ (265, 9) (262, 9)
301 301 (301, 9) ./data/ ./data/SI15/ (301, 9) (298, 9)
276 276 (276, 9) ./data/ ./data/SI16/ (276, 9) (273, 9)
267 267 (267, 9) ./data/ ./data/SI17/ (267, 9) (264, 9)
259 259 (259, 9) ./data/ ./data/SI18/ (259, 9) (256, 9)
260 260 (260, 9) ./data/ ./data/SI19/ (260, 9) (257, 9)
263 263 (263, 9) ./data/ ./data/SI20/ (263, 9) (260, 9)
260 260 (260, 9) ./data/ ./data/SI21/ (260, 9) (257, 9)
255 255 (255, 9) ./data/ ./data/SI22/ (255, 9) (

In [31]:
# Soujanya Chatterjee
# 2:06 PM (9 minutes ago)
	
# to me
import pandas as pd, numpy as np, os, csv, glob, math, matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
from datetime import datetime
from scipy.stats import *
import gzip
import pickle
from collections import Counter

def find_majority(k):
    myMap = {}
    maximum = ( '', 0 ) # (occurring element, occurrences)
    for n in k:
        if n in myMap: myMap[n] += 1
        else: myMap[n] = 1

        # Keep track of maximum on the go
        if myMap[n] > maximum[1]: maximum = (n,myMap[n])

    return maximum[0]

# _dir = 'W:\\Students\\cstress_features\\data\\data\\SI02\\'

def decodeLabel(label):
    label = label[:2]  # Only the first 2 characters designate the label code

    mapping = {'c1': 0, 'c2': 1, 'c3': 1, 'c4': 0, 'c5': 0, 'c6': 0, 'c7': 2}

    return mapping[label]

def readstressmarks(participantID, filename):
    features = []
    for file in os.listdir(filename):    
        if file.endswith("marks.txt.gz"):        
            with gzip.open(os.path.join(filename, file), 'r') as file:
                for line in file.readlines():
                    line = line.decode('utf8').strip()
                    parts = [x.strip() for x in line.split(',')]                    
                    label = parts[0][:2]  
                    if label not in ['c7','c6']:
                        stressClass = decodeLabel(label)
                        features.append([participantID, stressClass, int(parts[2]), int(parts[3])])
    return np.array(features)

_dirr = './data/'
parti = np.array(os.listdir(_dirr) )
print(parti)
header = ['participant','starttime','endtime','label','f_1','f_2','f_3','f_4','f_5','f_6','f_7']
fea_cols = ['f_1','f_2','f_3','f_4','f_5','f_6','f_7']
data = []
for p in parti:
    if p in ['feature.csv','feature_ecg.csv','feature_rip.csv','feature_rip_ecg.csv','feature_all.csv','SI05','SI09','SI11','SI14','SI23','SI24','.ipynb_checkpoints']:
        continue
    else:
        if os.path.isdir(os.path.join(_dirr,p)):
           
            _dir = (os.path.join(_dirr,p))
            gt_marks = readstressmarks(p,_dir)
            groundtruth = pd.DataFrame({'participant': gt_marks[:, 0], 'label': gt_marks[:, 1], 'starttime': gt_marks[:, 2],
                                        'endtime': gt_marks[:, 3]}, columns=['participant','label','starttime','endtime'])
            groundtruth = groundtruth.sort_values('starttime')
   
            x = []
            for file in os.listdir(_dir):    
                    if file.endswith("22norm.p"):                    
                        with open(_dir+'/'+file, 'rb') as f:  
                            x = pickle.load(f)
            if len(x)==0:
                continue
#             print(x)
            dataset = pd.DataFrame({'starttime': x[:, 0], 'endtime': x[:, 1], 'f_1': x[:, 2]
                                   , 'f_2': x[:, 3], 'f_3': x[:, 4], 'f_4': x[:, 5]
                                   , 'f_5': x[:, 6], 'f_6': x[:, 7], 'f_7': x[:, 8]}, columns=['starttime','endtime','f_1','f_2',
                                                                                               'f_3','f_4','f_5','f_6','f_7'])
           
            dataset = dataset.sort_values('starttime')

            for gt in range(len(dataset)):
                starttime = int(dataset['starttime'].iloc[gt])
                endtime = int(dataset['endtime'].iloc[gt])
                result = []
                for line in range(len(groundtruth)):
                    id, gtt, st, et = [groundtruth['participant'].iloc[line], groundtruth['label'].iloc[line], int(groundtruth['starttime'].iloc[line]),
                                      int(groundtruth['endtime'].iloc[line])]
                    if starttime < st:
                        continue
                    else:
                        if (starttime > st) and (endtime < et):
                            result.append(gtt)
                        if result:
                            fea = list(dataset[fea_cols].iloc[gt])
                            inter_data = [p, st,et,find_majority(result)],(fea)
                            flatten = lambda l: [item for sublist in l for item in sublist]

                            data.append(flatten(inter_data))
    #         print(data)
df = pd.DataFrame(data)
df.fillna(df.mean(),inplace=True)
df.to_csv(_dirr + '/' + 'feature_ecg_norm.csv', index=False, header=header)
df.shape

['.DS_Store' 'SI01' 'SI02' 'SI03' 'SI04' 'SI05' 'SI06' 'SI07' 'SI08'
 'SI09' 'SI10' 'SI11' 'SI12' 'SI13' 'SI14' 'SI15' 'SI16' 'SI17' 'SI18'
 'SI19' 'SI20' 'SI21' 'SI22' 'SI23' 'SI24' '.ipynb_checkpoints'
 'feature.csv' 'feature_rip.csv' 'feature_ecg.csv' 'feature_all.csv'
 'feature_rip_ecg.csv' 'feature_ecg_norm.csv']


(2597, 11)

In [32]:
feature_file = './data/feature_ecg_norm.csv'
feature = pd.read_csv(feature_file).values
y = np.int64(feature[:,3])
X = feature[:,4:]
print(X.shape,y.shape,np.sum(y))
groups = feature[:,0]

(2597, 7) (2597,) 493


In [33]:
from sklearn.decomposition import PCA
from pprint import pprint
from sklearn.metrics import f1_score
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC
m = len(np.where(y==0)[0])
n = len(np.where(y>0)[0])
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
import itertools
from sklearn.model_selection import ParameterGrid, cross_val_predict, GroupKFold,GridSearchCV
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from joblib import Parallel,delayed

delta = 0.1

paramGrid = {'kernel': ['rbf'],
             'C': np.logspace(.1,4,5),
             'gamma': [np.power(2,np.float(x)) for x in np.arange(-4, 4, .5)],
             'class_weight': [{0: w, 1: 1 - w} for w in [.4,.3,.2]],
             'probability':[False]
}
# clf = Pipeline([('rf', SVC())])
clf = SVC()
gkf = GroupKFold(n_splits=len(np.unique(groups)))
grid_search = GridSearchCV(clf, paramGrid, n_jobs=-1,cv=list(gkf.split(X,y,groups=groups)),
                           scoring='f1_weighted',verbose=5)
grid_search.fit(X,y)

print("Best parameter (CV score=%0.3f):" % grid_search.best_score_)
print(grid_search.best_params_)

import warnings
from sklearn.metrics import classification_report
warnings.filterwarnings('ignore')
clf = grid_search.best_estimator_
y_pred = cross_val_predict(clf,X,y,cv=gkf.split(X,y,groups=groups))
print(confusion_matrix(y,y_pred),classification_report(y,y_pred))

Fitting 18 folds for each of 240 candidates, totalling 4320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 354 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 552 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 786 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 1056 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 1362 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done 1704 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 2082 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 2496 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done 2946 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done 3432 tasks      | elapsed:   27.0s
[Parallel(n_jobs=-1)]: Done 3954 tasks      | elapsed:   48.1s
[Parallel(n_jobs=-1)]: Done 4320 out of 4320 | elapsed:   58.1s finished


Best parameter (CV score=0.913):
{'C': 11.885022274370183, 'class_weight': {0: 0.4, 1: 0.6}, 'gamma': 0.1767766952966369, 'kernel': 'rbf', 'probability': False}
[[2005   99]
 [ 125  368]]               precision    recall  f1-score   support

           0       0.94      0.95      0.95      2104
           1       0.79      0.75      0.77       493

    accuracy                           0.91      2597
   macro avg       0.86      0.85      0.86      2597
weighted avg       0.91      0.91      0.91      2597



In [34]:
import pickle
clf.set_params(probability=True)
print(clf)
clf.fit(X,y)
pickle.dump(clf,open('/home/jupyter/mullah/moods/stress_model.p','wb'))

SVC(C=11.885022274370183, break_ties=False, cache_size=200,
    class_weight={0: 0.4, 1: 0.6}, coef0=0.0, decision_function_shape='ovr',
    degree=3, gamma=0.1767766952966369, kernel='rbf', max_iter=-1,
    probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)


In [ ]:
import pickle
clf.set_params(probability=True)
print(clf)
clf.fit(X,y)
pickle.dump(clf,open('/home/jupyter/mullah/cc3/ecg_model_feature_standardization.p','wb'))

In [ ]:
from sklearn_porter import Porter
porter = Porter(clf, language='java')
output = porter.export(export_data=True)
# print(output)
text_file = open("SVM1.java", "w")
text_file.write(output)
text_file.close()
print(clf.probA_,clf.probB_)

In [ ]:
from sklearn_porter import Porter